In [163]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import keras 
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback
from pdb import set_trace as bp


#TESTA SEN!!!!
#chord.chord.show()


In [166]:
class Bithoven:
    def __init__(self,seq_len=100, net_in=np.array([]), net_out=np.array([]), n_voc=0, w=0,n=[]):
        self.ns = seq_len     #number of sequences
        self.network_input = net_in
        self.network_output = net_out
        self.n_vocab = n_voc
        self.model = Sequential()
        self.weights = w
        self.notes = n
        self.song_len = 500
        
        self.prediction_output=[]
        self.pitchnames = 0
        self.int_from_note= 0
        
        
    def get_data(self):
        
        for track in glob.glob("midi_songs/*.mid"):
            midi_track = converter.parse(track)
            to_be_parsed = 0
          
            different_instruments = instrument.partitionByInstrument(midi_track)

            if different_instruments:
                to_be_parsed = different_instruments.parts[0].recurse()
            else:
                to_be_parsed = midi_track.flat.notes

            for el in to_be_parsed:
                if isinstance(el, note.Note):
                    self.notes.append(str(el.pitch))
                elif isinstance(el,chord.Chord):
                    self.notes.append('.'.join(str(a_note) for a_note in el.normalOrder))
         

#                     for a_note in el.normalOrder:
#                         self.notes.append('.'.join(str(a_note)))
        
        self.pitchnames = sorted(set(item for item in self.notes)) #GÖR OM TILL KLASSISK FOR-LOOP
        self.int_from_note = dict((note,number) for number,note in enumerate(self.pitchnames))
        
                        
        self.n_vocab = len(set(self.notes)) #Number of pitchnames 
        
 

    
    
    def prep_data(self):
        
        self.network_output = []
        self.network_input = np.zeros((1*(len(self.notes)-self.ns),self.ns,1))
#         bp()

        test_count = [];
        for i in range(0,len(self.notes)-self.ns,1):
             
            for j,characters in enumerate(self.notes[i:i+self.ns]):
                test_count.append(0)
#                 bp()
                self.network_input[i,j,0]= self.int_from_note[characters]
            self.network_output.append(self.int_from_note[ self.notes[i+self.ns] ])

        self.network_input /= self.n_vocab


        with open('data/notes', 'wb') as the_file:
#             bp()
            pickle.dump(self.notes,the_file)
        
    def create_rnn(self):
        
        self.model.add(LSTM(128,input_shape = (self.ns,1), return_sequences=True))

        self.model.add(Dropout(0.3))
        self.model.add(LSTM(128))
        self.model.add(Dense(96))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(self.n_vocab))
        self.model.add(Activation('softmax'))
        self.model.compile(loss='categorical_crossentropy',optimizer='rmsprop')
        

            
        
    def train_rnn(self):
        
        the_file = "weights-improvement-{epoch:02d}--{loss:.4f}-bigger.hdf5"
        cpoint = ModelCheckpoint(the_file, monitor = 'loss', verbose=0,save_best_only = True, mode = 'min')
       # print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(self.model.layers[0].get_weights()))
        cb_list = [cpoint] #callbacks
    
        inp = self.network_input
        out = np_utils.to_categorical(self.network_output)

        self.model.fit(inp, out, epochs = 15, batch_size = 64, callbacks = cb_list)
        
        
        
        """ 
        
        NEDAN BÖRJAR GENERATE MUSIC  
        
        """
        
    def gen(self, ready_notes=False):
        if ready_notes:
            with open('data/notes','rb') as the_file:
                notes = pickle.load(the_file)
        
        else:
            pass

   
        self.prep_data()
        unnormalized = self.network_input*self.n_vocab        
        
        self.model.load_weights('weights.hdf5')
        #print(self.model.layers[0].get_weights())
        
        
        
        self.create_music()
        self.gen_midi()
    
    def gen_midi(self):
        pause = 0
        notes_out = []
        
        """
        
        nn - new note
        nc - new chord
        
        """
        
        for pattern in self.prediction_output:
            
            if('.' in pattern) or pattern.isdigit():
                #if we enter here, we know that the pattern is a chord
                notes_in_chord = pattern.split('.')
                notes =[]
                
                for cur_note in notes_in_chord:
                    nn = note.Note(int(cur_note))
                    nn.storedInstrument = instrument.Piano()
                    notes.append(nn)
                    
                nc = chord.Chord(notes)
                nc.offset = pause
                notes_out.append(nc)
                
            else:
                #if we enter here, we know that the pattern is a note
                nn = note.Note(pattern)
                nn.offset=pause
                nn.storedInstrument = instrument.Piano()
                notes_out.append(nn)
                
            pause += 1.0/2
        
        stream_of_midi = stream.Stream(notes_out)
        stream_of_midi.write('midi', fp='generated_song.mid')
                

    
    def create_music(self):
        #langd=self.network_input.shape[0]*self.network_input.shape[1]
        #tempvec=np.reshape(self.network_input,(1,langd))
        
       # print(tempvec.shape)
        start = np.random.randint(0, len(self.network_input)-1)
        pattern = self.network_input[start]
        prediction_output = []

        note_from_int = dict((num, note) for num, note in enumerate(self.pitchnames))
    
        indices=[]
        
        for j in range(self.song_len):
            
            predicted_in = np.reshape(pattern, (1,len(pattern),1 ) )
            p = self.model.predict(predicted_in, verbose = 0)
            ind = np.argmax(p)
            indices.append(ind)
            self.prediction_output.append(note_from_int[ind])
            pattern = np.append(pattern, ind)
            pattern = pattern[1:len(pattern)]
            
        
    def main_train(self):
        self.get_data()
        print('Prepare data')
        self.prep_data()
        print('Create RNN')
        self.create_rnn()
        print('Försöker hitta fel')
        print(self.network_input)
        print('start trainning')
        self.train_rnn()
        print(np.bincount(self.network_output))
        
    def main_generate(self):
        self.gen()
        
            
        

In [167]:
"""
RUN THIS CELL TO TRAIN THE NETWORK

AND GENERATE MUSIC

"""
#print('Program start')
generator = Bithoven()
generator.main_train()    
#generator.get_data()
#generator.prep_data()

prompt = input("BEFORE YOU GO ON, CHANGE A FILENAME TO weights.hdf5. THEN PRESS ENTER.")

generator.main_generate()

#print(generator.ns)
#print(len(generator.notes))
#for layer in generator.model.layers:
#    weights = layer.get_weights() # list of numpy arrays


Prepare data
Create RNN
Försöker hitta fel
[[[0.93073593]
  [0.58008658]
  [0.80952381]
  ...
  [0.93939394]
  [0.80952381]
  [0.81818182]]

 [[0.58008658]
  [0.80952381]
  [0.58008658]
  ...
  [0.80952381]
  [0.81818182]
  [0.87012987]]

 [[0.80952381]
  [0.58008658]
  [0.46320346]
  ...
  [0.81818182]
  [0.87012987]
  [0.84415584]]

 ...

 [[0.93506494]
  [0.86580087]
  [0.93073593]
  ...
  [0.82683983]
  [0.7965368 ]
  [0.82683983]]

 [[0.86580087]
  [0.93073593]
  [0.87012987]
  ...
  [0.7965368 ]
  [0.82683983]
  [0.2987013 ]]

 [[0.93073593]
  [0.87012987]
  [0.2987013 ]
  ...
  [0.82683983]
  [0.2987013 ]
  [0.93073593]]]
start trainning
Epoch 1/15
9290/9290 [==============================] - 54s 6ms/step - loss: 4.8748
Epoch 2/15
9290/9290 [==============================] - 48s 5ms/step - loss: 4.6426
Epoch 3/15
9290/9290 [==============================] - 47s 5ms/step - loss: 4.5154
Epoch 4/15
9290/9290 [==============================] - 48s 5ms/step - loss: 4.3896
Epoch 5/15


<_io.TextIOWrapper name='weights.hdf5' mode='r' encoding='UTF-8'>
